In [1]:
import os
import sys
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from pathlib import Path
import timm
from scipy.signal import butter, sosfiltfilt

if bool(os.environ.get("KAGGLE_URL_BASE", "")):
    tcn_source_path = '/kaggle/input/pytorch-tcn-library/pytorch_tcn-1.2.3/'
    sys.path.append(tcn_source_path)
from pytorch_tcn import TCN

if bool(os.environ.get("KAGGLE_URL_BASE", "")):
    print("Running on Kaggle, adjusting paths...")
    sys.path.insert(0, "/kaggle/input/hsm-source-files")
else:
    print("Running locally, adjusting paths...")
    sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..")))

from src.utils.utils import get_raw_data_dir, get_models_save_path, running_in_kaggle
from src.utils.constants import Constants
from src.utils.eeg_spectrogram_creator import EEGSpectrogramGenerator
from src.utils.feature_extraction import FeatureExtraction


class InfCFG:
    """Configuration for inference."""
    
    DATA_PATH = get_raw_data_dir()
    N_SPLITS = 5
    SEED = 42
    
    DATA_PREP_VOTE_METHOD = "sum_and_normalize" 

    if running_in_kaggle():
        TEST_EEG_SPEC_PATH = '/kaggle/working/test_eeg_spectrograms/'
        TCN_MODEL_BASE_DIR = f'/kaggle/input/tcn-sum-votes'
        CNN_MODEL_BASE_DIR = f'/kaggle/input//multicnn-sum-votes'
        HEAD_MODEL_BASE_DIR = f'/kaggle/input/multi-modal-heads'
    else:
        TEST_EEG_SPEC_PATH = DATA_PATH / 'custom_eegs' / 'test_cwt'
        MODEL_BASE_PATH = get_models_save_path()
        TCN_MODEL_BASE_DIR = MODEL_BASE_PATH / "TCNModel" / DATA_PREP_VOTE_METHOD
        CNN_MODEL_BASE_DIR = MODEL_BASE_PATH / "MultiSpectCNN" / DATA_PREP_VOTE_METHOD
        HEAD_MODEL_BASE_DIR = MODEL_BASE_PATH / "MultiModalHead" / DATA_PREP_VOTE_METHOD

    TCN_NUM_CHANNELS = 20
    TCN_CHANNEL_SIZES = [64, 128, 128, 256, 256, 512, 512, 512]
    TCN_KERNEL_SIZE = 21
    TCN_DROPOUT = 0.35
    TCN_DOWNSAMPLE_FACTOR = 3
    TCN_EMBED_SIZE = TCN_CHANNEL_SIZES[-1]

    CNN_MODEL_NAME = 'tf_efficientnet_b0_ns'
    CNN_IN_CHANNELS = 8
    CNN_IMG_SIZE = (128, 256)
    CNN_EMBED_SIZE = 1280
    
    
    HEAD_HIDDEN_SIZE = 256
    HEAD_DROPOUT = 0.4
    TARGET_SIZE = 6

    INFERENCE_BATCH_SIZE = 64
    NUM_WORKERS = 0 

Path(InfCFG.TEST_EEG_SPEC_PATH).mkdir(parents=True, exist_ok=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
set_seed(InfCFG.SEED)



class ClassifierHead(nn.Module):
    def __init__(self, tcn_input_size, cnn_input_size, hidden_size, output_size, dropout):
        super().__init__()
        
        # 1. Create a separate BatchNorm for each feature type
        self.tcn_bn = nn.BatchNorm1d(tcn_input_size)
        self.cnn_bn = nn.BatchNorm1d(cnn_input_size)
        # -----------------------------------------------
        
        self.tcn_input_size = tcn_input_size
        self.cnn_input_size = cnn_input_size
        
        total_input_size = tcn_input_size + cnn_input_size
        
        self.fc1 = nn.Linear(total_input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # x is the concatenated tensor
        
        # 1. Split the input back into its TCN and CNN parts
        x_tcn, x_cnn = torch.split(x, [self.tcn_input_size, self.cnn_input_size], dim=1)
        
        # 2. Normalize each part *independently*
        x_tcn = self.tcn_bn(x_tcn)
        x_cnn = self.cnn_bn(x_cnn)
        
        # 3. Re-concatenate the (now separately normalized) features
        x = torch.cat([x_tcn, x_cnn], dim=1)
        
        # 4. Proceed with the rest of the model
        x = self.fc1(x); x = self.bn1(x); x = self.relu(x)
        x = self.dropout(x); x = self.fc2(x)
        return x

class FeatureDataset(Dataset):
    """ The dataset from your training script, used to feed features to the head. """
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)
    def __len__(self): return len(self.features)
    def __getitem__(self, idx): return self.features[idx], self.labels[idx]


def run_inference():
    """
    Runs the full inference pipeline:
    1. Generates spectrograms for the test set.
    2. Creates test dataloaders for EEG and Spectrograms.
    3. For each fold:
        a. Extracts TCN features.
        b. Extracts CNN features.
        c. Concatenates features.
        d. Runs the ClassifierHead on the combined features.
    4. Ensembles predictions and saves to submission.csv.
    """
    print(f"Using device: {device}")
    
    test_df = pd.read_csv(InfCFG.DATA_PATH / "test.csv")
    targets = Constants.TARGETS
    
    print("\nGenerating EEG spectrograms for the test set...")
    test_eeg_save_path = Path(InfCFG.TEST_EEG_SPEC_PATH)
    
    existing_specs = len(list(test_eeg_save_path.glob("*.npy")))
    if existing_specs == len(test_df['eeg_id'].unique()):
        print("Test EEG spectrograms already exist. Skipping generation.")
    else:
        spectrogram_creator = EEGSpectrogramGenerator(["cwt"])
        test_eeg_ids = test_df["eeg_id"].unique()
        
        for eeg_id in tqdm(test_eeg_ids, desc="Generating Test EEG Specs"):
            eeg_path = InfCFG.DATA_PATH / "test_eegs" / f"{eeg_id}.parquet"
            eeg = pd.read_parquet(eeg_path)
            spectrograms = spectrogram_creator.generate(eeg)
            np.save(test_eeg_save_path / f"{eeg_id}.npy", spectrograms['cwt'])

    print("\nCreating test dataloaders for feature extraction...")
    extractor = FeatureExtraction(InfCFG, device)

    eeg_loader_test = extractor.get_eeg_inference_loader(
        test_df, InfCFG.DATA_PATH, InfCFG.TCN_DOWNSAMPLE_FACTOR
    )
    
    spec_loader_test = extractor.get_spec_inference_loader(
        test_df, targets, InfCFG.DATA_PATH, InfCFG.CNN_IMG_SIZE, InfCFG.TEST_EEG_SPEC_PATH
    )

    all_fold_predictions = []

    for fold_k in range(InfCFG.N_SPLITS):
        print("\n" + "="*50)
        print(f"Processing Fold {fold_k}")
        print("="*50)

        print(f"Loading TCN Model for Fold {fold_k}...")
        tcn_model_path = Path(InfCFG.TCN_MODEL_BASE_DIR) / f'best_model_fold{fold_k}.pth'
        if not tcn_model_path.exists():
            print(f"Warning: TCN Model not found at {tcn_model_path}. Skipping fold.")
            continue
            
        tcn_extractor = extractor.build_tcn_feature_extractor(tcn_model_path)
        
        tcn_features_test, _ = extractor.extract_features(
            tcn_extractor, eeg_loader_test
        )
        del tcn_extractor; torch.cuda.empty_cache()
        print(f"TCN Features shape: {tcn_features_test.shape}")

        print(f"Loading CNN Model for Fold {fold_k}...")
        cnn_model_path = Path(InfCFG.CNN_MODEL_BASE_DIR) / f'best_model_fold{fold_k}.pth'
        if not cnn_model_path.exists():
            print(f"Warning: CNN Model not found at {cnn_model_path}. Skipping fold.")
            continue
            
        cnn_extractor = extractor.build_cnn_feature_extractor(cnn_model_path)
        
        cnn_features_test, _ = extractor.extract_features(
            cnn_extractor, spec_loader_test
        )
        del cnn_extractor; torch.cuda.empty_cache()
        print(f"CNN Features shape: {cnn_features_test.shape}")

        combined_features_test = np.concatenate([tcn_features_test, cnn_features_test], axis=1)
        input_size = combined_features_test.shape[1]
        print(f"Combined Features shape: {combined_features_test.shape}")
        
        print(f"Loading Head Model for Fold {fold_k}...")
        head_model_path = Path(InfCFG.HEAD_MODEL_BASE_DIR) / f'best_head_fold{fold_k}.pth'
        if not head_model_path.exists():
            print(f"Warning: Head Model not found at {head_model_path}. Skipping fold.")
            continue
            
        head_model = ClassifierHead(
            tcn_input_size=InfCFG.TCN_EMBED_SIZE,  # <-- Use new config var
            cnn_input_size=InfCFG.CNN_EMBED_SIZE,  # <-- Use new config var
            hidden_size=InfCFG.HEAD_HIDDEN_SIZE,   # <-- This is now 256
            output_size=InfCFG.TARGET_SIZE,
            dropout=InfCFG.HEAD_DROPOUT
        ).to(device)
        head_model.load_state_dict(torch.load(head_model_path, map_location=device))
        head_model.eval()

        dummy_labels = np.zeros((len(combined_features_test), InfCFG.TARGET_SIZE))
        test_feat_dataset = FeatureDataset(combined_features_test, dummy_labels)
        test_feat_loader = DataLoader(
            test_feat_dataset, 
            batch_size=InfCFG.INFERENCE_BATCH_SIZE, 
            shuffle=False, 
            num_workers=InfCFG.NUM_WORKERS
        )

        current_fold_preds = []
        with torch.no_grad():
            for features, _ in tqdm(test_feat_loader, desc=f"Predicting Head Fold {fold_k}"):
                features = features.to(device)
                outputs = head_model(features)
                probs = F.softmax(outputs, dim=1).cpu().numpy()
                current_fold_preds.append(probs)
        
        all_fold_predictions.append(np.concatenate(current_fold_preds))


    if not all_fold_predictions:
        print("No models were found or predictions were generated. Aborting.")
        return

    print("\nEnsembling predictions across all folds...")
    avg_predictions = np.mean(all_fold_predictions, axis=0)
    
    print("Creating submission file...")
    submission_df = pd.DataFrame({'eeg_id': test_df['eeg_id'].unique()})
    submission_df[targets] = avg_predictions
    submission_df.to_csv('submission.csv', index=False)
    
    print("\nInference complete. submission.csv created.")
    print(submission_df.head())

if __name__ == '__main__':
    run_inference()

2025-11-07 15:34:40,043 :: root :: INFO :: Initialising Utils
2025-11-07 15:34:40,085 :: root :: INFO :: Initialising Datasets
2025-11-07 15:34:40,086 :: root :: INFO :: Initialising Models


Running locally, adjusting paths...
Using device: cuda

Generating EEG spectrograms for the test set...
Test EEG spectrograms already exist. Skipping generation.

Creating test dataloaders for feature extraction...

Processing Fold 0
Loading TCN Model for Fold 0...


Extracting Features:   0%|          | 0/1 [00:00<?, ?it/s]

TCN Features shape: (1, 512)
Loading CNN Model for Fold 0...


/home/maiko/miniconda3/envs/aicomp/lib/python3.13/site-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b0_ns to current tf_efficientnet_b0.ns_jft_in1k.
  model = create_fn(


...

/home/maiko/miniconda3/envs/aicomp/lib/python3.13/site-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b0_ns to current tf_efficientnet_b0.ns_jft_in1k.
  model = create_fn(


Extracting Features:   0%|          | 0/1 [00:00<?, ?it/s]

CNN Features shape: (1, 1280)
Combined Features shape: (1, 1792)
Loading Head Model for Fold 4...


Predicting Head Fold 4:   0%|          | 0/1 [00:00<?, ?it/s]


Ensembling predictions across all folds...
Creating submission file...

Inference complete. submission.csv created.
       eeg_id  seizure_vote  lpd_vote  gpd_vote  lrda_vote  grda_vote  \
0  3911565283      0.116667  0.040531  0.002931   0.203282    0.03735   

   other_vote  
0     0.59924  
